# Test Equation

$$
u' = -100 \cdot u + 10 \cdot \cos(t)
$$

In [28]:
from rk_methods import *
from imex_methods import *
from measure import*
from order_estimate import *

def g(t, u):
    return -100 * u

G = [[-100]]

def f(t, u):
    return 10 * np.cos(t)

def f_exact(t):
    return 1/10001 * (9001 * np.exp(-100 * t) + 10 * np.sin(t) + 1000 * np.cos(t))

Ns = list(range(500, 8001, 50))

# IMEX Euler

$$
    y_{n+1} = y_n + h ( f_1(t_n, y_n) + f_2(t_{n + 1}, y_{n+1}))
$$

In [29]:
A = [[1]]
b = [1]
c = [1]
A_ = [[0, 0], [1, 0]]
b_ = [1, 0]

table = create_mesasurement(f, g, A, A_, b, b_, c, 0, 20, f_exact, [0.2, 0.1, 0.05], G, Ns, verbose=False)

from IPython.display import HTML, display
display(HTML(table))

/home/peti/elte/msc/num_mod_project/proj/imex-runge-kutta-methods/rk_methods.py:110: RuntimeWarning: The iteration is not making good progress, as measured by the 
 improvement from the last ten iterations.
  k = np.reshape(fsolve(lambda k: calc_k(k) - k, base), (s, n))


Name,Exec time (s),Step Size,Error,Exec time (s),Step Size,Error,Exec time (s),Step Size,Error
ERK,0.0533,0.00741,0.19575,0.08544,0.00471,0.0989,0.13517,0.00268,0.04989
IRK,0.08573,0.04,0.1635,0.28011,0.00833,0.09977,0.79307,0.00342,0.04977
IMEX,0.03761,0.04,0.16357,0.17481,0.00833,0.09977,0.44764,0.00342,0.04977
IMEX Lin,0.02213,0.04,0.16357,0.10273,0.00833,0.09977,0.35068,0.00342,0.04977
IMEX LU,0.03265,0.04,0.16357,0.19893,0.00833,0.09977,0.4809,0.00342,0.04977


In [30]:
IMEX_E_order_result = order_from_fine_grid(lambda y0, Tl, Tr, N: IMEX(f, LinearImplicitSolver(G), y0, A, A_, b, b_, c, Tl, Tr, N), f_exact(0), 0, 20, [2**11, 2**12, 2**13], 2**16)
print('Estimating order from fine grid:')
print(f'Order result: {IMEX_E_order_result}')

Estimating order from fine grid:
Order result: [0.83634028 0.97674382]


# IMEX trapezoid

$$
y_{n+1} = y_n + \tfrac{h}{2} (f(t_n, y_n) + f_1(t_{n+1}, y_n + h \cdot f(t_n, y_n)) + f_2(t_{n+1}, y_{n+1}))
$$

In [31]:
from imex_methods import *

h, t, y = IMEX_trapezoid(f, g, f_exact(0), 0, 20, 4000)

error = np.linalg.norm(y - f_exact(t), np.inf)

IMEX_trapezoid_order_result = order_from_fine_grid(lambda y0, Tl, Tr, N: IMEX_trapezoid(f, g, y0, Tl, Tr, N), f_exact(0), 0, 20, [2**11, 2**12, 2**13], 2**15)
print('Estimating order from fine grid:')
print(f'Order result: {IMEX_trapezoid_order_result}')

/home/peti/elte/msc/num_mod_project/proj/imex-runge-kutta-methods/imex_methods.py:154: RuntimeWarning: The iteration is not making good progress, as measured by the 
 improvement from the last ten iterations.
  y[i + 1] = fsolve(f_impl, yn)


Estimating order from fine grid:
Order result: [2.14374279 2.09936475]


# IMEX midpoint

In [32]:
A = [[1/2]]
b = [1]
c = [1/2]
A_ = [[0, 0], [1/2, 0]]
b_ = [0, 1]

table = create_mesasurement(f, g, A, A_, b, b_, c, 0, 20, f_exact, [0.2, 0.1, 0.05, 0.025], G, Ns, verbose=False)

from IPython.display import HTML, display
display(HTML(table))

Name,Exec time (s),Step Size,Error,Exec time (s),Step Size,Error,Exec time (s),Step Size,Error,Exec time (s),Step Size,Error
ERK,0.04527,0.01176,0.18649,0.03337,0.0093,0.09717,0.06646,0.00727,0.04956,0.15575,0.00541,0.02472
IRK,0.08076,0.02667,0.1911,0.13825,0.01739,0.09531,0.27604,0.01212,0.04705,0.40196,0.00889,0.02385
IMEX,0.05267,0.02667,0.19111,0.11698,0.01739,0.09532,0.1522,0.01212,0.04705,0.19391,0.00889,0.02385
IMEX Lin,0.03026,0.02667,0.19111,0.04602,0.01739,0.09532,0.09398,0.01212,0.04705,0.10674,0.00889,0.02385
IMEX LU,0.07012,0.02667,0.19111,0.13067,0.01739,0.09532,0.15437,0.01212,0.04705,0.2297,0.00889,0.02385


In [33]:
IMEX_midpoint_order_result = order_from_fine_grid(lambda y0, Tl, Tr, N: IMEX(f, LinearImplicitSolver(G), y0, A, A_, b, b_, c, Tl, Tr, N), f_exact(0), 0, 20, [2**11, 2**12, 2**13], 2**15)
print('Estimating order from fine grid:')
print(f'Order result: {IMEX_trapezoid_order_result}')

Estimating order from fine grid:
Order result: [2.14374279 2.09936475]


# IMEX3

In [34]:
gamma = (3 + np.sqrt(3)) / 6

A = [[gamma, 0], [1 - 2 * gamma, gamma]]
b = [1/2, 1/2]
c = [gamma, 1 - gamma]
A_ = [[0, 0, 0], [gamma, 0, 0], [gamma - 1, 2*(1 - gamma), 0]]
b_ = [0, 1/2, 1/2]

table = create_mesasurement(f, g, A, A_, b, b_, c, 0, 20, f_exact, [0.2, 0.1, 0.05], G, Ns, verbose=False)

from IPython.display import HTML, display
display(HTML(table))

Name,Exec time (s),Step Size,Error,Exec time (s),Step Size,Error,Exec time (s),Step Size,Error
ERK,0.07083,0.016,0.1841,0.05044,0.01333,0.09279,0.15791,0.01111,0.04648
IRK,0.20396,0.03636,0.19946,0.25812,0.02222,0.09344,0.31047,0.016,0.04835
IMEX,0.09238,0.03636,0.19948,0.22096,0.02222,0.09345,0.19716,0.016,0.04835
IMEX Lin,0.04732,0.03636,0.19948,0.13785,0.02222,0.09345,0.118,0.016,0.04835
IMEX LU,0.09664,0.03636,0.19948,0.66041,0.02222,0.09345,0.21483,0.016,0.04835


In [35]:
IMEX_3_order_result = order_from_fine_grid(lambda y0, Tl, Tr, N: IMEX(f, LinearImplicitSolver(G), y0, A, A_, b, b_, c, Tl, Tr, N), f_exact(0), 0, 20, [2**13, 2**14, 2**15], 2**17)
print('Estimating order from fine grid:')
print(f'Order result: {IMEX_3_order_result}')

Estimating order from fine grid:
Order result: [2.84630441 2.93412842]
